# Implementing the "Fohn DA" method on AWS18 data in python with `Pandas`
#### And exploring some basics of data exploration in python/Jupyter more broadly.

## Overview

Here we'll use the python `Pandas` data analysis libriary instead of Excel to implement the Fohn DA method on AWS18 data.

We'll start with a more broad introduction of using python for geospatial analyses.

We'll see how easy it is to:
1. Read CSV/text datasets
2. generate summary statistics
3. Sub-sample data in time
4. Create quality plots of data
5. Implement some basic programming logic to classify and flag data
    
## Intent
Get your feet wet with Python because:
* It's useful for seeing that there are other, perhaps easier/quicker/better ways of exploring data.
* It could help you with A4-A6 in this class.
* It could be relevant to your research (in and out of this class).
* Working with data is a fundamental, transferrable skill (beyond this class and beyond Penn State).

### Acknowledgements and learning resources

* Intro/background material was mostly pulled from the excellent presentation of [David Shean (UW)](https://dshean.github.io/) during the [2020 ICESat-2 Hackweek](https://github.com/ICESAT-2HackWeek/2020_ICESat-2_Hackweek_Tutorials/blob/master/05.Geospatial_Analysis)
    * I **highly** reccommend this and his [presentation available on Youtube](https://www.youtube.com/watch?v=46vxJYqUMsM&t)
* CU EarthLab's Earth Analytics Python Course: https://www.earthdatascience.org/courses/earth-analytics-python/
* Geohackweek: https://geohackweek.github.io/
* ICESat-2 hackweek: https://icesat-2hackweek.github.io/
* Zhuolai's use of `Pandas` in his A3. 

### Data/methods references
AWS18 data were produced by Jakobs et al 2020:
> Jakobs, C. L., Reijmer, C. H., Smeets, C. J. P. P., Trusel, L. D., van de Berg, W. J., van den Broeke, M. R., & van Wessem, J. M. (2020). A benchmark dataset of in situ Antarctic surface melt rates and energy balance. Journal of Glaciology, 66(256), 291–302. https://doi.org/10.1017/jog.2020.6

* These (and other AWS) data are freely-available here: https://doi.pangaea.de/10.1594/PANGAEA.910480
    * Note: The data in this notebook just have an extra decimal date column and are in CSV rather than text. If using the raw data from Pangaea, there shouldn't need to be any modifications to the code other than changing the file name.

The FöhnDA method for detecting fohn-induced melting was introduced by Laffin et al 2021:
> Laffin, M. K., Zender, C. S., Singh, S., Van Wessem, J. M., Smeets, C. J. P. P., & Reijmer, C. H. (2021). Climatology and Evolution of the Antarctic Peninsula Föhn Wind‐Induced Melt Regime From 1979–2018. Journal of Geophysical Research: Atmospheres, 126(4). https://doi.org/10.1029/2020JD033682
* Here we use a slightly modified version of their method as described later on in this notebook. 



## The big picture: the scientific Python landscape

* Python
* Jupyter/iPython
* NumPy, Pandas, Matplotlib, SciPy
* xarray, scikit-learn

One (aging) interpretation of this stack:

![2017 Scientific Python Stack](https://devopedia.org/images/article/60/7938.1587985662.jpg)  
Slide from Jake VanderPlas’s presentation at PyCon 2017, entitled “The Unexpected Effectiveness of Python in Science.”

## The geospatial Python landscape
* [GDAL](https://gdal.org/), [GEOS](https://trac.osgeo.org/geos), [PROJ](https://proj.org/)
* [rasterio](https://rasterio.readthedocs.io/en/latest/), [fiona](https://pypi.org/project/Fiona/), [shapely](https://pypi.org/project/Shapely/), [pyproj](https://pypi.org/project/pyproj/)
* [geopandas](https://geopandas.org/), [cartopy](https://scitools.org.uk/cartopy/docs/latest/), [xarray](http://xarray.pydata.org/en/stable)

## Pandas

`Pandas` is an incredibly useful data science toolkit, regardless of what your specific applications are. 

Think of it as a free, more efficient, more elegant, more replicable replacement for Excel.

>pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with "relational" or "labeled" data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python. Additionally, it has the broader goal of becoming the most powerful and flexible open source data analysis / manipulation tool available in any language. It is already well on its way towards this goal.

https://github.com/pandas-dev/pandas#main-features

If you are working with tabular data (rows and columns, like a csv or spreadsheet), especially time series data, `pandas` is a good solution.
* A better way to deal with tabular data, built on top of NumPy arrays
* With NumPy, you need to remember which column number (e.g., 3, 4) represented each variable (date, t2m, rh, etc)
* Pandas allows you to store data with different types, and then reference using more meaningful labels
    * NumPy: `aws18_df[:,2]`
    * Pandas: `aws18_df['TTT [°C] (at 2m height']`
* A good "10-minute" reference with examples: https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

If you are working with more complex data, like collections of tabular time series data from 100s of met stations or netCDF model output, you can use [`xarray` package](http://xarray.pydata.org/en/stable/), which extends the `pandas` data model to n-dimensions.
* Note: we used/are using `xarray` when reading RACMO netCDF data in A3 and A4. 
* A good "45-minute" introduction from OceanHackWeek 2020: https://xarray-contrib.github.io/xarray-tutorial/oceanhackweek-2020/xarray-oceanhackweek20.html

## What does the AWS18 data look like?

In [ ]:
# First, tell Jupyter where to find the csv data:
aws18_datafile = './Data/IMAU_aws18_high-res_meteo_hourly.csv'

We can use command line functions using a `!` within Jupyter notebooks, like:

In [ ]:
# use the command line utility 'head' to look at the csv file:
!head $aws18_datafile

## Let's use pandas to explore AWS18 data and classify fohn vs non-fohn melting

### First, we'll need to import the `Pandas` package and some others if we want to make some nice looking plots. 

In [ ]:
# Import the pandas package
import pandas as pd
pd.options.mode.chained_assignment = None  # turn off a specific type of warning

# as well as some other plotting-related packages we'll want to use.

# matplotlib allows plotting
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

# seaborn adds some extra visual appeal to our plots
import seaborn as sns

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# set some universal plot settings here
plt.rcParams["figure.dpi"] = 200
plt.rcParams['axes.xmargin'] = 0.05
sns.set_style('darkgrid')
sns.set_context("notebook", font_scale=0.75)
%config InlineBackend.figure_format = 'retina' # make high res plots for retina 5k displays

### Now that `Pandas` is imported, we can call it and its functions using `pd` in our notebook:

Let's use it here to read the CSV file:

In [ ]:
aws18_df = pd.read_csv(aws18_datafile)

### Already easier than Excel! Now let's take a look at the data:

In [ ]:
# what does the top of this dataframe object look like?
aws18_df.head()

In [ ]:
# What about the end?
aws18_df.tail()

### Easy! How about we generate some basic summary information for the data:

In [ ]:
aws18_df.describe()

### But what about those "..." columns?

For extra wide datasets, we need to tell pandas to show us all the data, if that's what we want:

In [ ]:
pd.set_option('display.max_columns', None)
aws18_df.describe()

From here on out, we'll get all columns!

### We can also call matplotlib's plot function to get a quick look at the data

Here, we've imported matplotlib as `plt`, so we can call its plot function as follows:

In [ ]:
plt.plot(aws18_df['TTT [°C] (at 2m height)'])

## What are conditions like when melt is happening?

We can select only the data when melt is occurring according to the SEB model, and show the descriptive statistics for these hours quite easily.

### To do this, we want to use the `pandas.DataFrame.loc` function:

* Docs here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html
* Good examples here: https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/date-time-types-in-pandas-python/subset-time-series-data-python/

In [ ]:
aws18_df.loc[aws18_df['Melt rate [mm w.e.] (surface melt, within dt)']>0].describe()

### What if we were interested in subsetting by time? 

We'd want to tell `Pandas` how to index the data -- in this case, we can use the `Date/Time` column in the data. 

In [ ]:
# read the data again, but tell it where the date
aws18_df = pd.read_csv(aws18_datafile, parse_dates=['Date/Time'], index_col=['Date/Time'])

In [ ]:
## Let's take another look at the DataFrame now:
aws18_df.head()

### We can see that the first field from the CSV is not a column now, but instead it has replaced the sequential index column when we first read the file in using Pandas. 

We can query this index field as well:

In [ ]:
# View index values of dataframe
aws18_df.index

### Now that we've told Pandas where to read the date info, a simple plot of the data becomes more meaningful:

In [ ]:
plt.plot(aws18_df['TTT [°C] (at 2m height)'])

### We could also quickly spruce this up.

* Matplotlib example for a simple plot: https://matplotlib.org/stable/gallery/lines_bars_and_markers/simple_plot.html


In [ ]:
fig, ax = plt.subplots()
ax.plot(aws18_df['TTT [°C] (at 2m height)'], label='T2m')

ax.set(xlabel='Date', ylabel='Temperature [°C]',
       title='AWS18 data')

plt.show()

## Also very easy to add another data series to that same vertical axis:

In [ ]:
fig, ax = plt.subplots()
ax.plot(aws18_df['TTT [°C] (at 2m height)'], label='T2m')

# add a second data series and make it 50% transparent
ax.plot(aws18_df['Surf temp [°C] (modelled)'], label='Tsurf', alpha=0.5)

ax.set(xlabel='Date', ylabel='Temperature [°C]',
       title='AWS18 data')

# add a legend now that we have 2 lines
ax.legend()

plt.show()

This shows us how the skin (surface) temperature is limited to 0°C.

As a result, in summer there is typically a near-surface temperature inversion: temperature increases with height above the surface.
    
It's also interesting to see that air temperatures > 0°C are an imperfect indicator of surface melt. 
* Note the times that T2m > 0°C, yet the skin temperature is < 0 °C (i.e., not melting despite "warm" air)

## Indexing based on our Date/Time field is also useful because now we can select data based as a function of time using the `pandas.DataFrame.loc` function.

* Docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html

For example, what if we just wanted data for summer 2017/18 (December 2017, January 2018, February 2018)?    

In [ ]:
aws18_df.loc['2017-12':'2018-02']

### Let's get the descriptive stats now for that interval:

In [ ]:
aws18_df.loc['2017-12':'2018-02'].describe()

## How about selecting all Decembers? 

Pandas understands time operators, and since we have a date/time index, we can query for index values based on dates/times:

In [ ]:
aws18_df.loc[aws18_df.index.month==12]

### What if we wanted to select mutiple months, like all DJFs?

One way is to select data where the month index is equal to 12 or 1 or 2.

In python, `|` is a 'bitwise operator' meaning OR:
* https://www.w3schools.com/python/python_operators.asp

In [ ]:
# this creates a new dataframe containing a subset of aws18_df where the index month is in D,J, or F.
aws18_djf_df = aws18_df[(aws18_df.index.month==12) | (aws18_df.index.month==1) | (aws18_df.index.month==2)]
aws18_djf_df

In [ ]:
# what's it look like?
aws18_djf_df.describe()

### How's summer compare to winter?

Let's create a JJA subset and then difference the descriptive statistics:

In [ ]:
aws18_jja_df = aws18_df[(aws18_df.index.month==6) | (aws18_df.index.month==7) | (aws18_df.index.month==8)]
aws18_jja_df.describe()

In [ ]:
aws18_jja_df.describe() - aws18_djf_df.describe()

### So what have we learned from looking at AWS18 data so far?

It's colder in winter: on average 17.5°C colder. 

But the maximum temperature in JJA is quite high! +8.2°C (47°F) in the dark of the polar night! Only 3.4°C colder than the maximum temperature during summer. 

## The Fohn effect

We know from the literature that this region, and this AWS specficially is subject to warm, dry, and windy conditions when the fohn effect is active. 

Laffin and coauthors (https://doi.org/10.1029/2020JD033682) for example developed a method termed `FohnDA` to classify in AWS data when a fohn melt event is likely occurring:
1. T2m > 0°C
2. Relative humidity < 30th percentile
3. Wind speed > 60th percentile

We could implement this in Excel and assess fohn vs. non-fohn melting. In fact, we *have* done this! 

But wouldn't it be easier in python and pandas given how easy it is to select and summarize the data? Yes!

## FohnDA in `Pandas`:

Fundamentally, we just need to select locations (i.e., rows) where the three detection criteria are met. 

And since this assignment is most interested in melt, let's use `Surf temp [°C] (modelled)` = 0 instead of T2m > 0°C.

First, let's get the threshold values for humidity and wind speed:

In [ ]:
# relative humidity 30th percentile
rh_30th = aws18_df['RH [%] (at 2m height)'].quantile(0.3)
rh_30th

In [ ]:
# wind speed 80th percentile value:
ws_60th = aws18_df['ff [m/s] (at 10m height)'].quantile(0.6)
ws_60th

### Now we just need to find where the three variables are all met:

In [ ]:
aws18_df.loc[(aws18_df['Surf temp [°C] (modelled)']==0) & (aws18_df['RH [%] (at 2m height)']<rh_30th) & (aws18_df['ff [m/s] (at 10m height)']>ws_60th)]

### Create a column for fohn-induced melt

The above gives us the 1280 rows (hours) where fohn-induced melt is occurring, and we could do descriptive stats on this to get see what fohn melt conditions are like. 

But we also want to compare to non-fohn melt conditions, and perhaps more readily query fohn/non-fohn melt conditiosn in the future, so it'll be helpful if we create a new column in the aws18_df where we flag fohn melt first.

The following creates a new column where these three parameters are met. Note we're not using `loc` here:


In [ ]:
aws18_df['fohn_melt'] = (aws18_df['Surf temp [°C] (modelled)']==0) \
                        & (aws18_df['RH [%] (at 2m height)']<rh_30th) \
                        & (aws18_df['ff [m/s] (at 10m height)']>ws_60th)

Printing the DataFrame object out will show this as a new, boolean [True/False] column: 

In [ ]:
aws18_df

### For a sanity check, we can show just the locations where fohn_melt is true, and see that we find the same 1280 hours of fohn induced melt.

In [ ]:
aws18_df.loc[aws18_df['fohn_melt']==True]

### Now let's classify all other non-fohn melt

In [ ]:
aws18_df['non_fohn_melt'] = (aws18_df['fohn_melt']==False) \
                            & (aws18_df['Melt rate [mm w.e.] (surface melt, within dt)']>0) 

In [ ]:
aws18_df

### Great, now we can find the locations (rows/hours) fohn and non-fohn melt conditions and describe them:

In [ ]:
aws18_df.loc[aws18_df['fohn_melt']==True].describe()

In [ ]:
aws18_df.loc[aws18_df['non_fohn_melt']==True].describe()

### If we just wanted to get the counts:

In [ ]:
aws18_df['fohn_melt'].value_counts()

In [ ]:
aws18_df['non_fohn_melt'].value_counts()

### And we can define and query only the columns we're most interested in rather than looking at all data descriptions:

This creates a string array with the column names that we can then use to select:

In [ ]:
cols_of_interest = ['ff [m/s] (at 10m height)','RH [%] (at 2m height)','TTT [°C] (at 2m height)',
                    'Cloud cov [%]','Qh [W/m**2]','Qe [W/m**2]','Net SW [W/m**2]','Net LW [W/m**2]']

aws18_df[cols_of_interest].loc[aws18_df['fohn_melt']==True].describe()

In [ ]:
aws18_df[cols_of_interest].loc[aws18_df['non_fohn_melt']==True].describe()

## Now let's create variables that track cumulative fohn and non-fohn melt hours

* Need to implement this using some if/then type logic. 
* The basics are:
    * Create a new column that will hold a counter value for consecutive hours of melt
    * If a current hour is melting set the counter to 1 + what was in the previous counter value
    * So a single hour of melt = 1
    * if this is followed by another melt hour we get 1 + 1

In [ ]:
# first, create a blank column
aws18_df['fohn_consec_hours'] = 0

# for every row in the range of the length of the dataframe
# len gives the length (i.e., total number of data rows)
# range iterates through the rows (i.e., 0, 1, 2, 3 ... n)

for i in range(len(aws18_df)):
    # check if this row holds a fohn melt event
    if aws18_df['fohn_melt'][i]==True:
        # if true, in set the melt counter value for this row (hour) to 1 plus the value from the previous row (hour)
        aws18_df['fohn_consec_hours'][i]=1 + aws18_df['fohn_consec_hours'][i-1]
    else:
        # otherwise set the value in this row to zero (i.e, there's no melt)
        aws18_df['fohn_consec_hours'][i]=0

In [ ]:
# first, create a blank column
aws18_df['non_fohn_consec_hours'] = 0

for i in range(len(aws18_df)):
    # check if this row holds a non-fohn melt event
    if aws18_df['non_fohn_melt'][i]==True:
        # if true, in set the melt counter value for this row (hour) to 1 plus the value from the previous row (hour)
        aws18_df['non_fohn_consec_hours'][i]=1 + aws18_df['non_fohn_consec_hours'][i-1]
    else:
        # otherwise set the value in this row to zero (i.e, there's no melt)
        aws18_df['non_fohn_consec_hours'][i]=0

## Now we can get info about the maximum consecutive duration fohn and non-fohn melt events:

Looking the max values in the descriptive table will work:

In [ ]:
aws18_df.describe()

Or by just querying the max value:

In [ ]:
max_fohn_melt_hrs = aws18_df['fohn_consec_hours'].max()
max_nonfohn_melt_hrs = aws18_df['non_fohn_consec_hours'].max()

print("Maximum consecutive hours of fohn-induced melt: " + str(max_fohn_melt_hrs))
print("Maximum consecutive hours of non-fohn-induced melt: " + str(max_nonfohn_melt_hrs))

### But when did the longest fohn melt event occur? 

We can use `idxmax` which gives the index value of the maximum value.

* See: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html

>Return index of first occurrence of maximum over requested axis.

And we have Date/Time as our index, so it'll give us that:


In [ ]:
max_fohn_time = aws18_df['fohn_consec_hours'].idxmax()
max_fohn_time

In [ ]:
# Show us everything in that row
aws18_df.loc[max_fohn_time]

### But what about when it started?

This is definitely a bit trickier. It requires thinking programmatically and understanding/Googling some of the finer details of `pandas` indexing. 

In [ ]:
# get the numerical index value (i.e., row number) at the time of maximum fohn melt duration
max_index = aws18_df.index.get_loc(max_fohn_time)
max_index

This tell us the row number where this max fohn duration value occurs.

Now if we want to find when it started, we just need to subtract the number of fohn melt hours minus 1 from this row number.

Then we can see what's in this row (especially the index value, which we set to the Date/Time column)

In [ ]:
# length of event minus 1 hour will tell us when it started
# recall aws18_df['fohn_consec_hours'].max() tells us the max duration (50 hours, here)
n_hrs_previous = int(aws18_df['fohn_consec_hours'].max() - 1)

# now give the index (date/time) of when this was
aws18_df.index[max_index - n_hrs_previous]

Now we know when it started, we can also just see what else what happening then, and make sure it looks like we've selected the correct start time.

In [ ]:
aws18_df.loc['2016-05-25 09:00:00']

## Let's make a plot of all melt, and highlight fohn melt

### First, let's convert hourly data to daily max values using the `pandas.DataFrame.resample` function:
This type of resampling is incredibly easy! In fact, this is what I used when we were exploring daily A2 data -- I just resampled from hourly to daily. This sort of operation would be much, much more difficult and time consuming to implement in Excel, and much more prone to introducing errors.

Documentation here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html
> Resample time-series data.

> Convenience method for frequency conversion and resampling of time series. Object must have a datetime-like index (DatetimeIndex, PeriodIndex, or TimedeltaIndex), or pass datetime-like values to the on or level keyword.

In [ ]:
# convert to daily maxes
aws18_daily = aws18_df.resample('D').max()

## Now let's move on to making a nice looking plot to explore the timing of fohn-induced melt and its relation to melt rates.

Below, I've written some code that plots daily max melt data and shows the fohn-induced melt as vertical blue bars.

I've set plotting function up in a `for` loop to to produce a plot in every year that I've defined in a list of strings.

The plot has two axes -- one for the melt data, one for the Boolean True/False for fohn_melt, which in the plot convert to an integer field where 1=True. As such, I limit the axis with the fohn melt to between 0 and 1.

Both data series are plotted using the [`matplotlib.pyplot.fill_between`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.fill_between.html) method, which gives the step-line appearance as opposed to lines. This works especially well here as we want to shade the areas of fohn-induced melt. 

In [ ]:
# manually define an array of years as string values (i.e., in single quotes)
years = ['2015','2016','2017','2018']

for i in range(len(years)):
    
    # for the current iteration, i, reference the vale from the 'years' array we defined above
    year = years[i]
    
    # print out what year we're currently working with
    print(year)
    
    # Create figure and plot space
    fig, ax = plt.subplots(figsize=(15, 7.5))

    # subset in time based on the current year
    aws18_daily_sub = aws18_daily.loc[year+'-01':year+'-12']

    # shade areas where there's fohn-induced melt
    bg_steps = ax.fill_between(aws18_daily_sub.index.values,
                           aws18_daily_sub['fohn_melt'].astype(int),
                           label="FohnDA melt",
                           facecolor='steelblue',
                           step="mid",
                           linewidth=0,
                           alpha=0.6,
                           zorder=-1)

    # create a second y-axis to plot the melt data on
    ax2 = ax.twinx()
    melt_steps = ax2.fill_between(aws18_daily_sub.index.values,
                    aws18_daily_sub['Melt rate [mm w.e.] (surface melt, within dt)'],
                    label="Melt rate",
                    color="indianred",
                    step="mid",
                    zorder=1)

    # set some chart and axis properties
    ax.set_ylim((-0.05, 1))
    ax.yaxis.grid(False)
    ax.yaxis.set_ticklabels([])
    ax.yaxis.set_ticks([])
    ax2.yaxis.grid(False)
    ax2.yaxis.set_label_position("left")
    ax2.yaxis.tick_left()

    # Set title and labels for axes
    ax2.set(ylabel="Melt rate [mm w.e.]")

    # Define the date format
    date_form = DateFormatter("%b-%y")
    ax.xaxis.set_major_formatter(date_form)

    plt.show()